# Самурай без меча подобен самураю с мечом, но только без меча

Поэтому начнём с разновидностей словарей

## Содержание
* [Collections](#Collections)  
* [OrderedDict](#OrderedDict)    
* [defaultdict](#defaultdict)    
* [Counter](#Counter)
* [namedtuple](#namedtuple)
 - [Сравнение производительности dict vs namedtuple vs class object](#Сравнение производительности)
* [deque](#deque)

# <span style="color:green">Collections</span>
Модуль `сollections` реализует специальные типы данных, предоставляя альтернативу встренным типам `dict`, `list`, `set` и `tuples`. 
   
**Основные типы модуля `collections`**  
  
|  Тип данных collections | Описание                                                                       |
|-------------------------|--------------------------------------------------------------------------------|
| OrderDict               | Словарь (dict), который запоминает порядок добавления элементов                |
| defaultdict             | Словарь (dict), который вызывает функцию для заполнения пропущенных значений   |
| Counter                 | Словарь (dict) для подсчета хэшируемых объектов                                |
| namedtuple()            | Функция для создания кортежей (tuple) с именованными полями                    |
| deque                   | Двустороняя очередь на основе списка (list) с быстрым индексированием объектов |

# <span style="color:green">OrderedDict</span>

**`collections.OrderedDict`** - oбычный словарь, только помнит в каком порядке добавлялись элементы.

Основные методы:
 - `popitem(last=True)` возвращает и удаляет пару ключ-значение, которые были добавлены последними (`last`=True) или первыми (`last`=False)
 - `move_to_end(key, last=True)` - добавляет ключ в конец если `last`=True, и в начало, если `last`=False

**В целом, очень редко используется**

### Примеры использования

In [1]:
from collections import OrderedDict

**Создание упорядоченного словаря**

In [2]:
od = OrderedDict()
od['Data'] = 'Mining' # первый
od['In'] = 'Action' # второй
print(type(od))
od

<class 'collections.OrderedDict'>


OrderedDict([('Data', 'Mining'), ('In', 'Action')])

**Создание упорядоченного словаря из существующего**

In [3]:
simple_dict = {'Warren': 84,  'Bill': 90, 'Mark': 71, 'Jeff': 112} # обычный словарь

In [4]:
order_dict = OrderedDict(simple_dict)
order_dict['Larry'] = 50 # добавляет в конец
order_dict

OrderedDict([('Warren', 84),
             ('Bill', 90),
             ('Mark', 71),
             ('Jeff', 112),
             ('Larry', 50)])

**Создание упорядоченного словаря из существующего после его сортировки**

Можно задать порядок словарю, например по возрастанию ключей:

In [5]:
key_sorted_dict = OrderedDict(sorted(simple_dict.items(), key=lambda t: t[0]))
key_sorted_dict

OrderedDict([('Bill', 90), ('Jeff', 112), ('Mark', 71), ('Warren', 84)])

Или по возрастанию значений:

In [6]:
val_sorted_dict = OrderedDict(sorted(simple_dict.items(), key=lambda t: t[1]))
val_sorted_dict

OrderedDict([('Mark', 71), ('Warren', 84), ('Bill', 90), ('Jeff', 112)])

Или по длине ключа:

In [7]:
len_sorted_dict = OrderedDict(sorted(simple_dict.items(), key=lambda t: len(t[0])))
len_sorted_dict

OrderedDict([('Bill', 90), ('Mark', 71), ('Jeff', 112), ('Warren', 84)])

**Удаление последнего элемента**

In [8]:
len_sorted_dict.popitem()

('Warren', 84)

In [9]:
len_sorted_dict

OrderedDict([('Bill', 90), ('Mark', 71), ('Jeff', 112)])

**Перемещение существующего элемента в конец**

In [10]:
len_sorted_dict.move_to_end('Mark')
len_sorted_dict

OrderedDict([('Bill', 90), ('Jeff', 112), ('Mark', 71)])

# <span style="color:green">defaultdict</span>

**`collections.defaultdict`** - oбычный словарь, но со значением по умолчанию, которое можно задавать. 

**Создание**

Основной атрибут:
- default_factory - вызываемый объект для создания значения по умолчанию.

In [11]:
from collections import defaultdict

In [12]:
print(defaultdict(int))       # значение по умолчанию 0
print(defaultdict(float))     # значение по умолчанию 0.0
print(defaultdict(list))      # значение по умолчанию []
print(defaultdict(dict))      # значение по умолчанию {}
print(defaultdict(lambda: 7)) # значение по умолчанию 7

defaultdict(<class 'int'>, {})
defaultdict(<class 'float'>, {})
defaultdict(<class 'list'>, {})
defaultdict(<class 'dict'>, {})
defaultdict(<function <lambda> at 0x7fc32c555598>, {})


<span style="color:blue">Чем плохо так писать?</span>

In [13]:
# mutable type
default = {'id': 0}
ddict = defaultdict(lambda: default.copy())

In [14]:
a = {'id': 10}

In [15]:
b = a

In [16]:
b['id'] = 8

In [17]:
a

{'id': 8}

In [18]:
ddict['lol']['id'] = 8
print(ddict)
print(default)
ddict[1]['id'] = 'azaza'
print(ddict)
print(default)

defaultdict(<function <lambda> at 0x7fc32cdbb950>, {'lol': {'id': 8}})
{'id': 0}
defaultdict(<function <lambda> at 0x7fc32cdbb950>, {'lol': {'id': 8}, 1: {'id': 'azaza'}})
{'id': 0}


In [19]:
ddict = defaultdict(lambda: {'id': 0})

In [20]:
ddict['lol']['id'] = 8
print(ddict)
ddict[1]['id'] = 'azaza'
print(ddict)

defaultdict(<function <lambda> at 0x7fc32cdbbe18>, {'lol': {'id': 8}})
defaultdict(<function <lambda> at 0x7fc32cdbbe18>, {'lol': {'id': 8}, 1: {'id': 'azaza'}})


### Примеры использования

Группировка данных:

In [21]:
s = [('yellow', 1), ('blue', 2), ('yellow', 3), ('blue', 4), ('red', 1)] * 100000

In [22]:
%%time
d = defaultdict(list) # значение по умолчанию - пустой список []
for k, v in s:
    d[k].append(v)

# print(d)

CPU times: user 92.4 ms, sys: 3.38 ms, total: 95.7 ms
Wall time: 94.9 ms


С помощью стандарных методов реализация получается гораздо менее эффективной.   
<span style="color:blue">Как то же самое можно реализовать с помощью стандарных методов словаря?</span>

In [23]:
%%time
# попробуйте реализовать код выше, не используя defaultdict
d = dict()
for k, v in s:
    d.setdefault(k, [])
    d[k].append(v)
    
# print(d)

CPU times: user 215 ms, sys: 0 ns, total: 215 ms
Wall time: 214 ms


Счетчик (например, для подсчета количества встречаемости букв):

In [24]:
s = 'mississippi'
d = defaultdict(int) # значение по умолчанию - 0
for k in s:
     d[k] += 1
        
print(d)

defaultdict(<class 'int'>, {'m': 1, 'i': 4, 's': 4, 'p': 2})


Также можно использовать, когда лень писать код проверки наличия объекта в коллекции (например, в рекомендациях для новых товаров нет статистики появлений).

# <span style="color:green">Counter</span>

**`collections.Counter`** позволяет хранить частоты объектов, по сути `defaultdict(int)`, но предоставляет дополнительные методы для работы счётчиков за счёт ограничения на тип значений.

**Основные методы:**
 -  `elements()` - возвращает список элементов;
 -  `most_common(n)` - возвращает n наиболее часто встречающихся элементов, в порядке убывания встречаемости. Если n не указано, возвращаются все элементы;
 -  `subtract()` - вычитание.

**Создание**

In [25]:
from collections import Counter

Счетчик можно создавать на основе итерируемых обхектов (списки, строки, ...) или инициализировать из словаря:

In [26]:
c = Counter()                           # пустой счетчик
print(c)
c = Counter('mississippi')              # счетчик из строки
print(c)
c = Counter({'red': 4, 'blue': 2})      # счетчик из словаря
print(c)
c = Counter(cats=4, dogs=8)             # счетчик из keyword-аргументов
print(c)

Counter()
Counter({'i': 4, 's': 4, 'p': 2, 'm': 1})
Counter({'red': 4, 'blue': 2})
Counter({'dogs': 8, 'cats': 4})


In [27]:
c = Counter(['diapers', 'beer', 'beer'])      # счетчик из списка
print(c)
print(c['bread']) # для пропущенного элемента счетчик равен нулю
print(c)

Counter({'beer': 2, 'diapers': 1})
0
Counter({'beer': 2, 'diapers': 1})


**elements()**

In [28]:
c = Counter("""расскажите про покупки про какие про покупки 
               про покупки про покупки про покупочки свои""".split())
print(c)

Counter({'про': 6, 'покупки': 4, 'расскажите': 1, 'какие': 1, 'покупочки': 1, 'свои': 1})


In [29]:
sorted(c.elements())

['какие',
 'покупки',
 'покупки',
 'покупки',
 'покупки',
 'покупочки',
 'про',
 'про',
 'про',
 'про',
 'про',
 'про',
 'расскажите',
 'свои']

Метод `elements()` работает только с целыми значениями и игнорирует нулевые и отрицальные значения:

In [30]:
c = Counter(a=0, b=10, c=2, d=-3)
print(c)
print(list(c.elements()))

Counter({'b': 10, 'c': 2, 'a': 0, 'd': -3})
['b', 'b', 'b', 'b', 'b', 'b', 'b', 'b', 'b', 'b', 'c', 'c']


**most_common(n)**   
Примечание:
работает за $O(len(c) * \log n)$

In [31]:
c = Counter('pneumonoultramicroscopicsilicovolcanoconiosis')
print(c)

Counter({'o': 9, 'i': 6, 'c': 6, 'n': 4, 's': 4, 'l': 3, 'p': 2, 'u': 2, 'm': 2, 'r': 2, 'a': 2, 'e': 1, 't': 1, 'v': 1})


In [32]:
c.most_common(3) # топ-3

[('o', 9), ('i', 6), ('c', 6)]

**substract(cnt) и update(cnt)** 

In [33]:
c = Counter(a=4, b=2, c=0, d=-2)
d = Counter(a=1, b=2, c=3, d=4, e=3)
c.subtract(d) # c - d
print(c)
c.update(d)
print(c)

Counter({'a': 3, 'b': 0, 'c': -3, 'e': -3, 'd': -6})
Counter({'a': 4, 'b': 2, 'c': 0, 'e': 0, 'd': -2})


Можно передавать всё, от чего берётся Counter

In [34]:
c.update(a=4)
print(c)
c.update('aaaa')
print(c)
c.update({'a': 4})
print(c)

Counter({'a': 8, 'b': 2, 'c': 0, 'e': 0, 'd': -2})
Counter({'a': 12, 'b': 2, 'c': 0, 'e': 0, 'd': -2})
Counter({'a': 16, 'b': 2, 'c': 0, 'e': 0, 'd': -2})


Можно читерить и делать счётчики нецелыми

In [35]:
c = Counter(a=7.2, b=-2.7, c=0.5, d=-3.0, f=0)
print(c)
print(c.most_common(2))

Counter({'a': 7.2, 'c': 0.5, 'f': 0, 'b': -2.7, 'd': -3.0})
[('a', 7.2), ('c', 0.5)]


**Приведение типов**

In [36]:
print(sum(c.values()))                 # общая сумма значений
print(list(c))                         # список уникальных элементов
print(set(c))                          # множество элементов
print(dict(c))                         # обычный словарь
print(c.items())                       # перевод в список (elem, cnt) пар
print(Counter(dict([(1, 2), (3, 4)]))) # от списка пар (elem, cnt) к счетчику
n = 3
print(c.most_common()[:-n-1:-1])       # n наименее распостраненных элементов

2.0
['a', 'b', 'c', 'd', 'f']
{'c', 'b', 'f', 'a', 'd'}
{'a': 7.2, 'b': -2.7, 'c': 0.5, 'd': -3.0, 'f': 0}
dict_items([('a', 7.2), ('b', -2.7), ('c', 0.5), ('d', -3.0), ('f', 0)])
Counter({3: 4, 1: 2})
[('d', -3.0), ('b', -2.7), ('f', 0)]


**Унарные операции с счетчиком**

In [37]:
print(c)              
print(+c)             # только положительные значения
print(-c)             # отрицательные значения по модулю

Counter({'a': 7.2, 'c': 0.5, 'f': 0, 'b': -2.7, 'd': -3.0})
Counter({'a': 7.2, 'c': 0.5})
Counter({'d': 3.0, 'b': 2.7})


**Операции с счетчиками**

In [38]:
c = Counter(a=3, b=1)
d = Counter(a=1, b=2)
print(c + d)                       # сложение счетчиков:  c[x] + d[x]
print(c - d)                       # вычитание (сохраняются только положительные значения!)
print(c & d)                       # пересечение:  min(c[x], d[x]) 
print(c | d)                       # объединение:  max(c[x], d[x])

Counter({'a': 4, 'b': 3})
Counter({'a': 2})
Counter({'a': 1, 'b': 1})
Counter({'a': 3, 'b': 2})


### Примеры использования

Counter используют, как следует из названия, чтобы считать счётчики.  
Предположим у нас есть 6 записей о покупках пользователей на сайте:

In [39]:
visits = [{'id': '1',  'items': ['dress', 't-shirt', 'ring']}, 
          {'id': '10', 'items': ['scarf', 'mittens']}, 
          {'id': '4',  'items': ['dress', 'ring']}, 
          {'id': '2',  'items': ['t-shirt', 'scrarf', 'dress']}, 
          {'id': '4',  'items': ['scarf']}, 
          {'id': '1',  'items': ['dress', 'dress', 'scarf']}]

Посчитаем частоту встречи 'id':

In [40]:
occurences = Counter()
for visit in visits:
    occurences[visit['id']] += 1
print(occurences)

Counter({'1': 2, '4': 2, '10': 1, '2': 1})


Тоже самое, но короче с помощью спискового включения (list comprehension):

In [41]:
print(Counter(x['id'] for x in visits))

Counter({'1': 2, '4': 2, '10': 1, '2': 1})


<span style="color:blue">Определите топ-3 самых популярных товаров</span>

In [42]:
occurences = Counter()
for visit in visits:
    c = Counter(visit['items'])
    occurences += c
    
print(occurences.most_common(3))

[('dress', 5), ('scarf', 3), ('t-shirt', 2)]


In [43]:
from functools import reduce
reduce(lambda x, y: x + y, (Counter(x['items']) for x in visits)).most_common(3)

[('dress', 5), ('scarf', 3), ('t-shirt', 2)]

# <span style="color:green">namedtuple</span>

Класс `collections.namedtuple`, как следует из названия, это tuple, к элементам которого можно обращаться по имени. 
Нужен, когда важна читабельность кода и память.

Основные методы:
   - somenamedtuple._make(iterable) - создание namedtuble из итерируемого объекта
   - somenamedtuple._asdict() - создает OrederedDict по именам полей
   - somenamedtuple._replace() - возвращает новый namedtuple заменяя значение у соответвующего поля

**Простой пример**

In [44]:
from collections import namedtuple

In [45]:
# создание namedtuple
Point = namedtuple('Point', ['x', 'y'])
print(Point._fields)

Point = namedtuple('Point', 'x y')
print(Point._fields)

('x', 'y')
('x', 'y')


In [46]:
# создание объекта из из keyword-аргументов
p = Point(x=1, y=2)
print(p)

# передача аргументов с помощью словаря
p = Point(**{'x': 2, 'y': 3})
print(p)

print("p.x =", p.x)
print("p.y =", p.y)

Point(x=1, y=2)
Point(x=2, y=3)
p.x = 2
p.y = 3


In [47]:
p = p._make([10, 12])
print(p)

Point(x=10, y=12)


In [48]:
p._asdict() # переход к OrederedDict

OrderedDict([('x', 10), ('y', 12)])

In [49]:
p._replace(x=20) # замена значения

Point(x=20, y=12)

**Пример использования**

In [50]:
import random
import time
import sys

## <a id="Сравнение производительности">Сравнение производительности (dict vs namedtuple vs class object with slots)</a>

#### Память

In [51]:
# словарь словарей
objects_dict = {
    x: {
        'first_property': random.random(),
        'second_property': random.random(),
        'fourth_property': random.random(),
        'do_you_know_where_the_third_property_is': random.random() < 0.5,
        }
    for x in range(100000)
}

print('dict object size is', sys.getsizeof(objects_dict[0]))

dict object size is 240


In [52]:
# словарь namedtuple-ов
ComplexObject = namedtuple(
    'ComplexObject', 
    'first_property second_property fourth_property do_you_know_where_the_third_property_is'
)

objects_namedtuple = {
    k: ComplexObject(**v)
    for k, v in objects_dict.items()
}
print('namedtuple object size is', sys.getsizeof(objects_namedtuple[0]))

namedtuple object size is 80


In [53]:
class MObject(object):
    __slots__ = ('first_property', 'second_property', 'fourth_property', 'do_you_know_where_the_third_property_is')
    def __init__(self, first_property, second_property, fourth_property, do_you_know_where_the_third_property_is):
        self.first_property = first_property
        self.second_property = second_property
        self.fourth_property = fourth_property
        self.do_you_know_where_the_third_property_is = do_you_know_where_the_third_property_is
        
# словарь экзэмпляров класса        
objects_class = {
    k: MObject(**v)
    for k, v in objects_dict.items()
}
print('slots class object size is', sys.getsizeof(objects_namedtuple[0]))

slots class object size is 80


#### Время

**Dict**

In [54]:
%%time
total_time = 0.
first_value = 0.
second_value = 0.
fourth_value = 0.
bool_value = False
random.seed(42)
for _ in range(1000000):
    index = random.randint(0, 99999)
    obj = objects_dict[index]
    bool_value ^= obj['do_you_know_where_the_third_property_is'] # ^ - побитовое исключающее ИЛИ (xor)
    first_value += obj['first_property']
    second_value += obj['second_property']
    fourth_value += obj['fourth_property']
    
    start = time.time()
    obj['do_you_know_where_the_third_property_is']
    obj['first_property']
    obj['second_property']
    obj['fourth_property']
    total_time += time.time() - start
    
print("first_value =", first_value)
print("second_value =", second_value)
print("fourth_value =", fourth_value)
print("bool_value =", fourth_value)

print("Total time =",total_time)
print()

first_value = 500025.94019293465
second_value = 500910.65309275064
fourth_value = 500324.5613034676
bool_value = 500324.5613034676
Total time = 0.34126734733581543

CPU times: user 3.52 s, sys: 0 ns, total: 3.52 s
Wall time: 3.52 s


**Namedtuple**

In [55]:
%%time
total_time = 0.
first_value = 0.
second_value = 0.
fourth_value = 0.
bool_value = False
random.seed(42)
for _ in range(1000000):
    index = random.randint(0, 99999)
    obj = objects_namedtuple[index]
    bool_value ^= obj.do_you_know_where_the_third_property_is
    first_value += obj.first_property
    second_value += obj.second_property
    fourth_value += obj.fourth_property
    
    start = time.time()
    obj.do_you_know_where_the_third_property_is
    obj.first_property
    obj.second_property
    obj.fourth_property
    total_time += time.time() - start

print("first_value =", first_value)
print("second_value =", second_value)
print("fourth_value =", fourth_value)
print("bool_value =", fourth_value)

print("Total time =",total_time)
print()

first_value = 500025.94019293465
second_value = 500910.65309275064
fourth_value = 500324.5613034676
bool_value = 500324.5613034676
Total time = 0.5251772403717041

CPU times: user 4.47 s, sys: 2.87 ms, total: 4.47 s
Wall time: 4.48 s


**Class object**

In [56]:
%%time
total_time = 0.
first_value = 0.
second_value = 0.
fourth_value = 0.
bool_value = False
random.seed(42)
for _ in range(1000000):
    index = random.randint(0, 99999)
    obj = objects_class[index]
    bool_value ^= obj.do_you_know_where_the_third_property_is
    first_value += obj.first_property
    second_value += obj.second_property
    fourth_value += obj.fourth_property
    
    start = time.time()
    obj.do_you_know_where_the_third_property_is
    obj.first_property
    obj.second_property
    obj.fourth_property
    total_time += time.time() - start

print("first_value =", first_value)
print("second_value =", second_value)
print("fourth_value =", fourth_value)
print("bool_value =", fourth_value)

print("Total time =",total_time)
print()

first_value = 500025.94019293465
second_value = 500910.65309275064
fourth_value = 500324.5613034676
bool_value = 500324.5613034676
Total time = 0.37476468086242676

CPU times: user 3.69 s, sys: 3.89 ms, total: 3.69 s
Wall time: 3.69 s


### Мораль
Не используйте `namedtuple`, если нужна производительность, используйте слоты (`slots`) или словари (`dict`), но в последнем случае тот, кто будет читать ваш код, будет очень сильно вас не любить.

Чтобы не писать slots самому воспользуйтесь библиотекой [recordclass](https://pypi.org/project/recordclass/)

# <span style="color:green">deque</span>

`collections.deque(iterable, [maxlen])` - создаёт очередь из итерируемого объекта с максимальной длиной maxlen. Очереди очень похожи на списки (list), за исключением того, что добавлять и удалять элементы можно либо справа, либо слева.

Основные методы:
- append(x) - добавляет x в конец,
- appendleft(x) - добавляет x в начало,
- pop() - удаляет и возвращает последний элемент очереди,
- popleft() - удаляет и возвращает первый элемент очереди.

Так как курс не про алгоритмы, то на этом всё :)

#### Главное помнить, что индексирование быстрое, а также добавление, удаление в начало и в конец. Всё остальное медленное и лучше это не использовать. Только если очень хочется.

Примеры из документации:

In [57]:
from collections import deque

In [58]:
d = deque('ghi')                # создание deque из 3х эелементов
for elem in d:                  # итерирование по элементам 
    print(elem)

g
h
i


In [59]:
d.append('j')                   # добавление элемента в конец
d.appendleft('f')               # добавление элемента в начало
d                                

deque(['f', 'g', 'h', 'i', 'j'])

In [60]:
print(d.pop())                  # возвращает и удаляет последний элемент
print(d.popleft())              # возвращает и удаляет первый элемент
list(d)                          

j
f


['g', 'h', 'i']

In [61]:
# обращение к элементам по индексу
print(d[0])
print(d[-1])

g
i


In [62]:
list(reversed(d))              # разворот очереди

['i', 'h', 'g']

In [63]:
'h' in d

True

In [64]:
# расширение очереди к концу
d.extend('jkl')
d

deque(['g', 'h', 'i', 'j', 'k', 'l'])

In [65]:
# расширение очереди с начала
d.extendleft('abc')
d

deque(['c', 'b', 'a', 'g', 'h', 'i', 'j', 'k', 'l'])

In [66]:
# rotate(n) - последовательно переносит n элементов из начала в конец (если n отрицательно, то с конца в начало)
d.rotate(1) 
d

deque(['l', 'c', 'b', 'a', 'g', 'h', 'i', 'j', 'k'])

In [67]:
d.rotate(-1) 
d

deque(['c', 'b', 'a', 'g', 'h', 'i', 'j', 'k', 'l'])

In [68]:
d.clear()                        # очищение очереди

In [69]:
d

deque([])